# Lab 1 Answers:

### Exercise 2:

We were required to create a new folder in HDFS called textData. The command for this is given below:

In [ ]:
hadoop fs -mkdir textData

We then had to put the file textData.txt into this new folder on HDFS. This is done with the command:

In [ ]:
hadoop fs -put textData.txt textData/textData.txt

To confirm that the upload has been successful we can list the contents of the textData directory with the following command:

In [ ]:
hadoop fs -ls textData

### Exercise 3:

We are asked to list the contents of the w1e3-output folder on HDFS. This can be done with the following command:

In [ ]:
hadoop fs -ls w1e3-output

Upon running this command you should see something that looks like:

You should see from the output that only 1 part file has been produced here. This means that only one reducer process was executed here!

By viewing the contents of the results we can see that our map reduce job is sensitive to the precise punctuation following the words. E.g. it counts "Amen" and "Amen," separately. The output could have been improved by removing any punctuation marks from the keyspace.

### Exercise 4:

We include below the completed mapper.py file:

In [ ]:
#!/usr/bin/env python
import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
        # remove leading and trailing whitespace
        line = line.strip()

        # TODO: Students to output <lineLength, 1>
        # compute length of line using len function
        lineLength = len(line)

        # write the results to STDOUT (standard output);
        # what we output here will be the input for the 
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial lineLength count is 1
        print '%s\t%s' % (lineLength, 1)

We now include the completed reducer.py file:

In [ ]:
#!/usr/bin/env python
from operator import itemgetter
import sys

current_len = None
current_count = 0
vlen = None

# input comes from STDIN
for line in sys.stdin:
        # remove leading and trailing whitespace
        line = line.strip()

        # parse the input we got from mapper.py
        vlen, count = line.split('\t', 1)

        # convert count (currently a string) to int
        try:
                count = int(count)
        except ValueError:
                # count was not a number, so silently
                # ignore/discard this line
                continue

        # this IF-switch only works because Hadoop sorts map output
        # by key (here: vlen) before it is passed to the reducer
        if current_len == vlen:
                current_count += count
        else:
                if current_len:
                        # write result to STDOUT
                        # TODO: Students to produce <lineLength, count> output
                        print '%s\t%s' % (current_len, current_count)
                current_count = count
                current_len = vlen

# do not forget to output the last word if needed!
if current_len == vlen:
        # TODO: Students to produce <lineLength, count> output
        print '%s\t%s' % (current_len, current_count)

Command to execute the Hadoop job (note: USER must be replaced with your username):

In [ ]:
hadoop jar $HADOOP_STR/hadoop-streaming-2.7.0-mapr-1808.jar \
-libjars $HADOOP_STR/hadoop-streaming-2.7.0-mapr-1808.jar \
-input textData/* \
-output w1e4-output \
-mapper "python mapper.py" \
-file /home/USER/bd-sp-2017/w1e4/mapper.py \
-reducer "python reducer.py" \
-file /home/USER/bd-sp-2017/w1e4/reducer.py

Viewing the results we notice that asides from a line length of 0 the most frequent line length is 42.